# 用来将wav音频转成多普勒频移谱

### 对音频进行傅里叶变换，抹掉低频噪音后再重新生成wav

In [15]:
import wave as we
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy.signal as signal
import librosa
import librosa.display
import torch
import pickle

np.set_printoptions(suppress=True)

In [16]:
def read_wav1(path):
    #读出wav文件
    y, sr = librosa.load(path, sr=48000,mono=False)
    return y,sr

In [17]:
def fft_wav(waveData, framerate):
    #对waveData数据进行傅里叶变换
    f_array = np.fft.fft(waveData)
    axis_f = np.linspace(0, int(framerate/2), int(len(f_array)/2))
    return f_array

In [18]:
def transform_high_fram(wavefft,framerate,minf,plots=False):
    #抹掉低频信号
    step_hz=(framerate/2) / (len(wavefft) / 2)
    savewav = []
    for i in range(int(minf/step_hz)):
        savewav.append(0)
    for i in range(int(minf/step_hz),len(wavefft)-int(minf/step_hz)):
        savewav.append(wavefft[i])
    for i in range(len(wavefft)-int(minf/step_hz),len(wavefft)):
        savewav.append(0)
    return savewav

In [19]:
def ffti_wav(savewav,filename,sr):
    #傅里叶逆变换保存文件
    i_array = np.fft.ifft(savewav)
    save_wav = i_array.real.reshape(len(i_array))
    save_wav=np.asfortranarray(save_wav.astype(np.float32))
    librosa.output.write_wav(filename,save_wav,sr)

##### 主函数

In [20]:
actions = os.listdir('./wav/src_wav')

for action in actions:
    folder='./wav/src_wav/'+action
    print(action)
    if 'person8' not in action:
        continue
    os.mkdir('./wav/high_wav/'+action)
    for filename in os.listdir(folder):
        if 'wav' not in filename:
            continue
        waveData, sr= read_wav1(folder+'/'+filename)
        f_array = fft_wav(waveData, sr)
        savewav=transform_high_fram(f_array,sr,18000)
        ffti_wav(savewav,'./wav/high_wav/'+action+'/'+filename,sr)
        print(filename+' OK!')

person1
person2
person3
person4
person5
person6
person7
person8
1.wav OK!
10.wav OK!
11.wav OK!
2.wav OK!
3.wav OK!
4.wav OK!
5.wav OK!
6.wav OK!
7.wav OK!
8.wav OK!
9.wav OK!


### 对音频进行傅里叶变换，抹掉超声频率后再重新生成wav

In [21]:
def read_wav3(path):
    #读出wav文件
    y, sr = librosa.load(path, sr=None,mono=False)
    return y,sr

In [22]:
def transform_low_fram(wavefft,framerate,minf,plots=False):
    #抹掉高频信号
    step_hz=(framerate/2) / (len(wavefft) / 2)
    savewav = []
    for i in range(int(minf/step_hz)):
        savewav.append(wavefft[i])
    for i in range(int(minf/step_hz),len(wavefft)-int(minf/step_hz)):
        savewav.append(0)
    for i in range(len(wavefft)-int(minf/step_hz),len(wavefft)):
        savewav.append(wavefft[i])
    return savewav

In [23]:
actions = os.listdir('./wav/src_wav')

for action in actions:
    folder='./wav/src_wav/'+action
    print(action)
    if 'person8' not in action:
        continue
    os.mkdir('./wav/low_wav/'+action)
    for filename in os.listdir(folder):
        if 'wav' not in filename:
            continue
        waveData, framerate = read_wav3(folder+'/'+filename)
        f_array = fft_wav(waveData, framerate)
        savewav=transform_low_fram(f_array,framerate,18000)
        ffti_wav(savewav,'./wav/low_wav/'+action+'/'+filename,framerate)
        print(filename+' OK!')

person1
person2
person3
person4
person5
person6
person7
person8
1.wav OK!
10.wav OK!
11.wav OK!
2.wav OK!
3.wav OK!
4.wav OK!
5.wav OK!
6.wav OK!
7.wav OK!
8.wav OK!
9.wav OK!


### 对超声波的wav文件进行stft变换，生成多普勒频移谱

In [24]:
def read_wav2(path):
    #读取wav文件
    y, sr = librosa.load(path, sr=48000,mono=False)
    return y,sr

In [25]:
def plot_stft(y,sr,path):
    #进行stft变换
    nfft=int(sr*0.7)
    
    data = librosa.amplitude_to_db(np.abs(librosa.stft(y,n_fft=nfft,hop_length=int(nfft/30))), ref=np.max)
    plt.figure()

    librosa.display.specshow(data[12500:], sr=sr, x_axis='time', y_axis='linear')
    plt.tight_layout()
    plt.axis('off')

    plt.savefig(path,bbox_inches='tight', pad_inches=0.0)
    plt.close()

##### 主函数

In [26]:
actions = os.listdir('./wav/high_wav')
for action in actions:
    folder='./wav/high_wav/'+action
    if '8' not in action:
            continue
    os.mkdir('./model_data/Doppler_pic/'+action)
    print(action)
    for filename in os.listdir(folder):
        print(filename+' ok')
        wavaData, framerate = read_wav2(folder+'/'+filename)
        path='./model_data/Doppler_pic/'+action+'/'+filename[:len(filename)-4]+'.jpg'
        plot_stft(wavaData, framerate,path)

person8
1.wav ok
10.wav ok
11.wav ok
2.wav ok
3.wav ok
4.wav ok
5.wav ok
6.wav ok
7.wav ok
8.wav ok
9.wav ok
